# Word Seperation for RAVDESS

In [1]:
from google.colab import drive
import os
import sys
import pickle
import numpy as np
from collections import OrderedDict
drive.mount('/content/drive')
os.chdir("./drive/MyDrive/Project/Data")

Mounted at /content/drive


In [5]:
%%capture
!pip install SpeechRecognition
!apt-get install -y swig libpulse-dev
!pip install pocketsphinx
!pip install pydub
import pydub
from pydub import AudioSegment
!pip install --upgrade "ibm-watson>=5.1.0"
from ibm_watson import SpeechToTextV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import speech_recognition as sr  

import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

Offline Speech recogition 


In [ ]:


r = sr.Recognizer()


test = sr.AudioFile('./Actor_01/03-01-01-01-01-01-01.wav')

with test as source:
    # r.adjust_for_ambient_noise(source)
    audio = r.record(source)


output = r.recognize_sphinx(audio)
out2 = r.recognize_google(audio, show_all=True)
print(output)
print(out2)


does it talk him by the door
{'alternative': [{'transcript': 'kids are talking by the door', 'confidence': 0.89194101}, {'transcript': 'kids are talkin by the door'}], 'final': True}


In [ ]:
# get names of all files for conversion
wav_list = {} 
for i in range(1,25):
    actor = "Actor_%02d"% (i,)
    wav_list[actor] = os.listdir(f"Actor_%02d"% (i,))


This does not work as reliably as using online services - as shown above. The corect translation is in the alternatives, however this can't be trusted if using data where the text is unknown

# IBM SPEECH regogntion

In [ ]:
# test ibm serice is working
authenticator = IAMAuthenticator('')#add ibm key here 
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)
# https://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/d80d29c5-b7cb-4314-903c-93f15e895993

speech_to_text.set_service_url('https://api.us-south.speech-to-text.watson.cloud.ibm.com')


In [ ]:

# from https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjL1Jeak5nyAhUM6p4KHaodA0cQFnoECBAQAw&url=https%3A%2F%2Fcloud.ibm.com%2Fapidocs%2Fspeech-to-text&usg=AOvVaw2wBZJEgs_Gs8ZRQfIvxSrA
# note newer versions of ibm watson can break this 


authenticator = IAMAuthenticator('') #add ibm key here 
speech_to_text = SpeechToTextV1(
    authenticator=authenticator
)

speech_to_text.set_service_url('https://api.us-south.speech-to-text.watson.cloud.ibm.com')
test = None
class MyRecognizeCallback(RecognizeCallback):
    def __init__(self):
        RecognizeCallback.__init__(self)

    def on_data(self, data):
        global test
        # print(json.dumps(data, indent=2))
        test = json.dumps(data, indent=2)
    def on_error(self, error):
        print('Error received: {}'.format(error))

    def on_inactivity_timeout(self, error):
        print('Inactivity timeout: {}'.format(error))

myRecognizeCallback = MyRecognizeCallback()

def ibm_stt(path):
    with open(path,'rb') as audio_file:       
        audio_source = AudioSource(audio_file)
        speech_to_text.recognize_using_websocket(
            audio=audio_source,
            content_type='audio/wav',
            recognize_callback=myRecognizeCallback,
            model='en-US_BroadbandModel',
            max_alternatives=3,
            timestamps=True,
            profanity_filter=False)

# Testing one file 

In [ ]:

path = './Actor_01/03-01-01-01-01-02-01.wav'

ibm_stt(path)
# print(test)
words = json.loads(test)["results"][0]['alternatives'][0]['timestamps']
scentence = json.loads(test)["results"][0]['alternatives'][0]['transcript']
print(words)
print(scentence)

In [ ]:
# file = '03-01-01-01-01-01-01.wav'
file = '03-01-01-01-01-02-01.wav'
file_dict = {}
whole_dict = {}
word_added = {}
times = {}
whole_dict[file] = scentence
for word_info in words:
    # print(word_info)
    times = {}
    word = word_info[0]
    start_time = word_info[1]
    end_time = word_info[2]
    times["start"]=word_info[1]
    times["stop"]=word_info[2]
    word_added[word] = times
    file_dict[file] = word_added
    print(f"Word: {word}, start_time: {start_time}, end_time: {end_time}")


In [ ]:
for file in file_dict:
    for word in file_dict[file]:
        # print(word)
        # print(file)
        start = file_dict[file][word]['start'] * 1000 
        stop = file_dict[file][word]['stop'] * 1000 
        newAudio = AudioSegment.from_wav(f"Actor_01/{file}")
        newAudio = newAudio[start:stop]
        newAudio.export(f'../TestData/{word}.wav', format="wav") 

In [ ]:
# # get names of all files for conversion
# wav_list = {} 
# for i in range(1,25):
#     actor = "Actor_%02d"% (i,)
#     # os.listdir(f'./{actor}')
#     wav_list[actor] = os.listdir(f"Actor_%02d"% (i,))
# print(wav_list)
# for actor in wav_list:
#     for file in wav_list[actor]:
#         path = f'./{actor}/{file}'
#         ibm_stt(path)
#         words = json.loads(test)["results"][0]['alternatives'][0]['timestamps']
#         scentence = json.loads(test)["results"][0]['alternatives'][0]['transcript']
#         file_dict = {}
#         whole_dict = {}
#         word_added = {}
#         times = {}
#         whole_dict[file] = scentence
#         for word_info in words:
#             # print(word_info)
#             times = {}
#             word = word_info[0]
#             start_time = word_info[1]
#             end_time = word_info[2]
#             times["start"]=word_info[1]
#             times["stop"]=word_info[2]
#             word_added[word] = times
#             file_dict[file] = word_added
#         print(words)
#         print(scentence)
#     for file in file_dict:
#         for word in file_dict[file]:
#             # print(word)
#             # print(file)
#             start = file_dict[file][word]['start'] * 1000 
#             stop = file_dict[file][word]['stop'] * 1000 
#             newAudio = AudioSegment.from_wav(f"Actor_01/{file}")
#             newAudio = newAudio[start:stop]
#             newAudio.export(f'../TestData/{actor}{file}{word}.wav', format="wav") 


# Run IBM speech regognition for work splitting - gets data and timestamps then splits the words into an output file 

In [5]:
# get names of all files for conversion
wav_list = {} 
for i in range(1,25):
    actor = "Actor_%02d"% (i,)
    # os.listdir(f'./{actor}')
    da = os.listdir(f"Actor_%02d"% (i,))
    wav_list[actor] = []
    for k in da:
        if '.wav' in k:
            wav_list[actor]
            # labels.append(k[6:8])
            wav_list[actor].append(k) 

actor_dict = {}
for actor in wav_list:
    whole_dict = {}
    file_dict = {}
    for file in wav_list[actor]:
        path = f'./{actor}/{file}'
        ibm_stt(path)
        words = json.loads(test)["results"][0]['alternatives'][0]['timestamps']
        scentence = json.loads(test)["results"][0]['alternatives'][0]['transcript']
        
        # whole_dict = {}
        word_added = {}
        times = {}
        whole_dict[file] = scentence
        for word_info in words:
            # print(word_info)
            times = {}
            word = word_info[0]
            start_time = word_info[1]
            end_time = word_info[2]
            times["start"]=word_info[1]
            times["stop"]=word_info[2]
            word_added[word] = times
            file_dict[file] = word_added
        print(words)
        print(scentence)
    actor_dict[actor]=file_dict
    for file in file_dict:
        for word in file_dict[file]:
            # print(word)
            # print(file)
            start = file_dict[file][word]['start'] * 1000 
            stop = file_dict[file][word]['stop'] * 1000 
            newAudio = AudioSegment.from_wav(f"{actor}/{file}")
            newAudio = newAudio[start:stop]
            newAudio.export(f'../TestData/{actor}_{file[:-4]}_{word}_{start}_{stop}.wav', format="wav") # replace TestData with desired output folder


{'Actor_01': ['03-01-01-01-01-01-01.wav',
  '03-01-01-01-02-02-01.wav',
  '03-01-01-01-01-02-01.wav',
  '03-01-01-01-02-01-01.wav',
  '03-01-02-01-02-01-01.wav',
  '03-01-02-01-01-01-01.wav',
  '03-01-02-01-01-02-01.wav',
  '03-01-02-02-01-02-01.wav',
  '03-01-02-02-02-02-01.wav',
  '03-01-02-02-02-01-01.wav',
  '03-01-02-01-02-02-01.wav',
  '03-01-02-02-01-01-01.wav',
  '03-01-03-01-01-01-01.wav',
  '03-01-03-02-01-02-01.wav',
  '03-01-03-01-02-01-01.wav',
  '03-01-03-01-02-02-01.wav',
  '03-01-03-02-01-01-01.wav',
  '03-01-03-01-01-02-01.wav',
  '03-01-04-02-01-01-01.wav',
  '03-01-04-01-02-02-01.wav',
  '03-01-03-02-02-02-01.wav',
  '03-01-03-02-02-01-01.wav',
  '03-01-04-01-01-02-01.wav',
  '03-01-04-01-02-01-01.wav',
  '03-01-04-01-01-01-01.wav',
  '03-01-04-02-02-02-01.wav',
  '03-01-05-01-01-01-01.wav',
  '03-01-04-02-02-01-01.wav',
  '03-01-04-02-01-02-01.wav',
  '03-01-08-02-01-02-01.wav',
  '03-01-08-01-01-01-01.wav',
  '03-01-07-01-01-01-01.wav',
  '03-01-05-02-01-02-01.wav'

In [ ]:
file_dict

{'03-01-01-01-01-01-01.wav': {'are': {'start': 1.19, 'stop': 1.26},
  'by': {'start': 1.66, 'stop': 1.79},
  'door': {'start': 1.92, 'stop': 2.33},
  'kids': {'start': 0.95, 'stop': 1.19},
  'talking': {'start': 1.26, 'stop': 1.66},
  'the': {'start': 1.79, 'stop': 1.92}},
 '03-01-01-01-01-02-01.wav': {'are': {'start': 1.21, 'stop': 1.3},
  'by': {'start': 1.7, 'stop': 1.84},
  'door': {'start': 1.96, 'stop': 2.36},
  'kids': {'start': 0.93, 'stop': 1.21},
  'talking': {'start': 1.3, 'stop': 1.7},
  'the': {'start': 1.84, 'stop': 1.96}},
 '03-01-01-01-02-01-01.wav': {'are': {'start': 1.29, 'stop': 1.38},
  'by': {'start': 1.66, 'stop': 1.81},
  'dogs': {'start': 0.95, 'stop': 1.29},
  'door': {'start': 1.92, 'stop': 2.33},
  'sitting': {'start': 1.38, 'stop': 1.66},
  'the': {'start': 1.81, 'stop': 1.92}},
 '03-01-01-01-02-02-01.wav': {'are': {'start': 1.27, 'stop': 1.36},
  'by': {'start': 1.6, 'stop': 1.74},
  'dogs': {'start': 0.95, 'stop': 1.27},
  'door': {'start': 1.86, 'stop': 2

In [ ]:
with open('file_dict.pkl', 'wb') as handle:
    pickle.dump(file_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('filename.pkl', 'wb') as handle:
    pickle.dump(whole_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# with open('actor_dict.pkl', 'wb') as handle:
#     pickle.dump(actor_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
# with open('actor_dict.pkl', 'rb') as handle:
#     test = pickle.load(handle)
# print (file_dict == test)


False


{'Actor_01': {'03-01-01-01-01-01-01.wav': {'are': {'start': 1.19,
    'stop': 1.26},
   'by': {'start': 1.66, 'stop': 1.79},
   'door': {'start': 1.92, 'stop': 2.33},
   'kids': {'start': 0.95, 'stop': 1.19},
   'talking': {'start': 1.26, 'stop': 1.66},
   'the': {'start': 1.79, 'stop': 1.92}},
  '03-01-01-01-01-02-01.wav': {'are': {'start': 1.21, 'stop': 1.3},
   'by': {'start': 1.7, 'stop': 1.84},
   'door': {'start': 1.96, 'stop': 2.36},
   'kids': {'start': 0.93, 'stop': 1.21},
   'talking': {'start': 1.3, 'stop': 1.7},
   'the': {'start': 1.84, 'stop': 1.96}},
  '03-01-01-01-02-01-01.wav': {'are': {'start': 1.29, 'stop': 1.38},
   'by': {'start': 1.66, 'stop': 1.81},
   'dogs': {'start': 0.95, 'stop': 1.29},
   'door': {'start': 1.92, 'stop': 2.33},
   'sitting': {'start': 1.38, 'stop': 1.66},
   'the': {'start': 1.81, 'stop': 1.92}},
  '03-01-01-01-02-02-01.wav': {'are': {'start': 1.27, 'stop': 1.36},
   'by': {'start': 1.6, 'stop': 1.74},
   'dogs': {'start': 0.95, 'stop': 1.27}

In [ ]:
print(len(actor_dict))
a=0
for i in test:
    a+=len(wav_list[i])
    # print()
a

24


1440

In [ ]:
a=0
for i in wav_list:
    a+=len(wav_list[i])
    # print()
a

1440

In [ ]:
print(len(file_dict))

120


In [ ]:
file_dict

{'03-01-01-01-01-01-01.wav': {'are': {'start': 1.19, 'stop': 1.26},
  'by': {'start': 1.66, 'stop': 1.79},
  'door': {'start': 1.92, 'stop': 2.33},
  'kids': {'start': 0.95, 'stop': 1.19},
  'talking': {'start': 1.26, 'stop': 1.66},
  'the': {'start': 1.79, 'stop': 1.92}},
 '03-01-01-01-01-01-02.wav': {'are': {'start': 1.29, 'stop': 1.47},
  'by': {'start': 1.93, 'stop': 2.12},
  'door': {'start': 2.26, 'stop': 2.71},
  'kids': {'start': 0.96, 'stop': 1.29},
  'talking': {'start': 1.47, 'stop': 1.93},
  'the': {'start': 2.12, 'stop': 2.26}},
 '03-01-01-01-01-02-01.wav': {'are': {'start': 1.21, 'stop': 1.3},
  'by': {'start': 1.7, 'stop': 1.84},
  'door': {'start': 1.96, 'stop': 2.36},
  'kids': {'start': 0.93, 'stop': 1.21},
  'talking': {'start': 1.3, 'stop': 1.7},
  'the': {'start': 1.84, 'stop': 1.96}},
 '03-01-01-01-01-02-02.wav': {'are': {'start': 1.3, 'stop': 1.44},
  'by': {'start': 1.91, 'stop': 2.1},
  'door': {'start': 2.25, 'stop': 2.68},
  'kids': {'start': 0.97, 'stop': 1.